In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import sklearn
from sklearn.preprocessing import LabelEncoder
import os
import pyodbc

In [2]:
conexion = pyodbc.connect('Driver={SQL Server};'
'Server=DESKTOP-34T1EIQ;'
'Database=RepuestosWeb;'
'Trusted_Connection=yes;', timeout=5000)

In [3]:
df_Orders=pd.read_sql("select * from VW_OrdenesEncimaPromedio", conexion)
#se quita total orden debido a que totalencima promedio ya calcula si el total 
#orden esta arriba o abajo de la medicion que buscamos por lo que es incesario
df_Orders.drop('Total_Orden', axis=1, inplace=True)
df_Orders.head(10)

,NombreCiudad,Genero,ID_Parte,ID_Categoria,TotalEncimaPromedio
0,Greensboro,F,AC2801112CORE-ORIG.REC,5205,1
1,Atlanta,F,AC1249108,3415,0
2,Kansas,M,AC2519109N-ORIG.REC,4863,0
3,Aurora,F,AC1251102,3537,1
4,Baltimore,F,AC1321101,3479,1
5,St. Paul,F,AC1240119PP,3425,1
6,Little Rock,F,AC1241123C-ORIG.REC,3416,0
7,Bakersfield,F,AC2518111NIQ-ORIG.REC,4035,1
8,Oklahoma,F,AC2800110NIQ,2923,1
9,Aurora,F,AC1248114,1356,1


In [4]:
#almacenar las variables x y y
#esto es para poder tener las variables dependientes e indipendientes
#independientes variables de entradas que identifican los atributos de cada uno de los elementos
#variable dependiente  que son las salida que son las que dependen de las de entrada para que se cumplan

x=df_Orders.iloc[:, 0:4].values

y=df_Orders.iloc[:,-1].values
#esto se vuelve un arreglo

In [5]:
#aplicar encoder
#podermo codificar atriutos no numericos segun una escala

le = LabelEncoder()
x[:,0] = le.fit_transform(x[:,0])
x[:,1] = le.fit_transform(x[:,1])
x[:,2] = le.fit_transform(x[:,2])
x[:,3] = le.fit_transform(x[:,3])
x

array([[34, 0, 328, 3881],
       [5, 0, 137, 2657],
       [42, 1, 271, 3745],
       ...,
       [87, 1, 11895, 5359],
       [8, 0, 11898, 5705],
       [34, 0, 11900, 5692]], dtype=object)

In [6]:
#dividimos los datos para poder tener datos para entrenamiento y otros para test
#se usa sckikit learn para llamar a la funcion split y lo dividimos en 80/20
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x,y, test_size=0.20, random_state=0)


In [8]:
#en ciertos casoss las escalas entre los datos van a estar muy diferentes como por ejemplo 100 robos al años en comparacion de 10 muertes por años
## en estos casos es mejor llevarlo a todo a una misma escala por meido de promedios
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
x_train = sc.fit_transform(x_train)
x_test = sc.transform(x_test)
x_train
#estos numero serca de uno por lo que si estan abajo de uno estan abajo del promeido y si estan arrriba de uno estan arriba del promedio esto para poder llevarlo a una misma escala

array([[-0.00411038,  0.96806859,  0.52064235,  0.53820817],
       [ 1.6748242 ,  0.96806859, -0.98326992,  0.56285424],
       [ 1.06754999, -1.03298465,  0.2961432 , -1.64296905],
       ...,
       [ 0.99610597, -1.03298465,  0.93889529, -0.8989658 ],
       [ 0.10305566,  0.96806859,  1.5152258 , -0.39341329],
       [-0.0755544 ,  0.96806859, -1.08971849, -0.19747703]])

In [9]:
#crear modelo NB
from sklearn.naive_bayes import GaussianNB
classifier=GaussianNB()
classifier.fit(x_train,y_train)

GaussianNB(priors=None, var_smoothing=1e-09)

In [10]:
#predecimos la varible dependiente usando las variables independientes del 20%
y_pred = classifier.predict(x_test)
y_pred

array([0, 1, 1, ..., 0, 1, 0], dtype=int64)

In [11]:
#con esto podriamos comparar manualmente uno por uno pero es mejor hacer la matriz de confucion
y_test

array([0, 0, 0, ..., 1, 0, 1], dtype=int64)

In [12]:
#generarmos la matriz de confucion y precision
from sklearn.metrics import confusion_matrix,accuracy_score
cm = confusion_matrix(y_test, y_pred) #crea la matriz de confucion 
ac = accuracy_score(y_test,y_pred) #define la presicion

In [13]:
#matriz de confusion
cm

array([[242, 959],
       [221, 960]], dtype=int64)

In [14]:
#presicion
ac

0.5046179680940386